<a href="https://colab.research.google.com/github/itssofiarce/sagrada/blob/main/codigo/extraccion_creditos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!pip install selenium webdriver-manager flask

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,244 kB]
Get:13 https://developer.download.nvidia.com/compute/cud

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

In [ ]:
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
def get_credits(delay=5):
    """
    Obtener las stats, con selenium, de la página de BGG de Sagrada
    args:
        delay: int, tiempo de espera para que cargue la página
    returns:
        pd.DataFrame, tabla con las estadísticas.
    """

    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = "https://boardgamegeek.com/boardgame/199561/sagrada/credits"
        driver.get(url)

        print(f"⏳ Esperando {delay} segundos para que carguen los datos...")
        time.sleep(delay)

        stats = driver.find_element(By.CLASS_NAME, "global-body-content-primary.ng-scope")

        # Títulos
        titulos = stats.find_elements(By.CLASS_NAME, "panel-title")
        titulos = [titulo for titulo in titulos if titulo.tag_name == "h3"]
        titulos_texto = [titulo.text for titulo in titulos]

        # Paneles
        paneles = stats.find_elements(By.CLASS_NAME, "panel-body")

        dic_stadisticos = {}

        for titulo, panel in zip(titulos_texto, paneles):
            if titulo not in ["RATINGS BREAKDOWN"]:
                stats_items = panel.find_elements(By.CLASS_NAME, "outline-item")

                for item in stats_items:
                    try:
                        clave = item.find_element(By.CLASS_NAME, "outline-item-title").text.strip()
                        desc = item.find_element(By.CLASS_NAME, "outline-item-description")
                        valores_li = desc.find_elements(By.TAG_NAME, "li")

                        # Si hay <li>, usamos todos los valores
                        if valores_li:
                            valores = [li.text.strip() for li in valores_li]
                        else:
                            valores = [desc.text.strip()]  # Solo un valor

                        dic_stadisticos[clave] = valores

                    except:
                        continue

    finally:
        driver.quit()

    # Convertimos a DataFrame
    df = pd.DataFrame.from_dict(dic_stadisticos, orient='index').reset_index()
    df.columns = ['Credit'] + [f'Value {i+1}' for i in range(df.shape[1] - 1)]
    return df

In [ ]:
estadisticas = get_credits()
estadisticas.to_csv("sagrada_stats.csv", index=False)
estadisticas

⏳ Esperando 5 segundos para que carguen los datos...


,Credit,Value 1
0,Primary Name,Sagrada
1,Alternate Names,Саґрада\nСаграда\nซากราดา\nサグラダ\n聖家族大教堂 SAGRAD...
2,Year Released,2017
3,Designers,Adrian Adamescu\nDaryl Andrews
4,Solo Designer,N/A
5,Artist,Peter Wocken
6,Publishers,Floodgate Games\nCranio Creations\nCrowD Games...
7,Developer,N/A
8,Graphic Designer,N/A
9,Sculptor,N/A


## Extracion de relaciones


In [ ]:
creditos_csv = pd.read_csv('sagrada_stats.csv')

In [ ]:
relaciones = []
def agregar_relacion(sujeto1, relacion, sujeto2):
    relaciones.append({
        "SUJETO1": sujeto1,
        "RELACION": relacion,
        "SUJETO2": sujeto2
    })

In [ ]:
# Nombre del juego
juego = creditos_csv.iloc[0, 1]

In [ ]:
# Iteramos sobre cada fila para obtener las relaciones
for index, row in creditos_csv.iterrows():
    credit = row['Credit']
    value = row['Value 1']

    # Ignoramos las filas de 'year released' y 'Altername Names'
    if credit in ['Year Released', 'Alternate Names']:
        continue

    # Si el valor no es NaN o vacío, procesamos la relación
    if pd.notna(value) and value != "N/A":

        items = value.split('\n')

        # Usamos el nombre del 'Credit' como la relación
        relacion = credit

        # Agregar una relación por cada elemento en la lista
        for item in items:
            item = item.strip()  # Limpiar espacios extra
            if item:  # Asegurarse de no agregar relaciones vacías
                agregar_relacion(juego, relacion, item)


In [ ]:
# Crear el DataFrame con las relaciones
df_relaciones = pd.DataFrame(relaciones)

In [ ]:
# Mostrar las primeras relaciones para verificar
print(df_relaciones.head())
print(f"Total de relaciones encontradas: {len(df_relaciones)}")

# Guardar a un CSV
df_relaciones.to_csv("relaciones_sagrada_generadas.csv", index=False)

   SUJETO1      RELACION          SUJETO2
0  Sagrada  Primary Name          Sagrada
1  Sagrada     Designers  Adrian Adamescu
2  Sagrada     Designers    Daryl Andrews
3  Sagrada        Artist     Peter Wocken
4  Sagrada    Publishers  Floodgate Games
Total de relaciones encontradas: 49
